## Exercise 3

### Descrição do Problema

O objetivo deste exercício é desenvolver uma AEAD com "Tweakable Block Ciphers”", 
sendo a cifra usada do tipo AES-256 ou o ChaCha20.

Esta cifra será usada num canal privade de informação assíncrona com acordo de chaves.
A autenticação dos agentes será com:
 - “X448 key exchange”
 - “Ed448 Signing&Verification” 
 
... e deve haver uma fase de confirmação da chave acordada.


Ou seja,

Use o “package” Cryptography para
1. Implementar uma AEAD com “Tweakable Block Ciphers” conforme está descrito na última secção do texto +Capítulo 1: Primitivas Criptográficas Básicas.  A cifra por blocos primitiva, usada para gerar a “tweakable block cipher”, é o AES-256 ou o ChaCha20.
2. Use esta cifra para construir um canal privado de informação assíncrona com acordo de chaves feito com “X448 key exchange” e “Ed448 Signing&Verification” para autenticação  dos agentes. Deve incluir uma fase de confirmação da chave acordada.

In [5]:
#Imports necessários
import os
import asyncio
import random
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric.x448 import X448PrivateKey
from cryptography.hazmat.primitives.asymmetric.ed448 import Ed448PrivateKey

## Criação das chaves privadas/públicas

O acordo de chaves, como indicado no enunciado, deve ser feito com __"X448 key exchange"__.

Para tal é necessário que cada um dos agentes criem as suas próprias chaves (privada-pública).

Neste caso será usando a __curva448__, como 
apresentado na API:
- https://cryptography.io/en/latest/hazmat/primitives/asymmetric/x448/

A chave resultante será usada no decorrer da comunicação assíncrona entre agentes para __cifrar/decifrar__ os dados.

In [2]:
def generateKeys():
    # Generate private key for exchange
    prv_key = x448.X448PrivateKey.generate()
    
    # Generate public key thorugh private key
    pub_key = private_key.public_key()
    
    return prv_key, pub_key
    

## Criação das chaves partilhadas

Após ter as chaves de cada agente (privada/pública), podemos estabelecer
o _exchange_, i.e o acordo entre ambos agentes,
sobre um segredo partilhado.

A _shared_key_ criada, tal como recomendado na API, deve ser passada por uma função de derivação, no âmbito de a tornar mais segura; adicionando mais informações à chave para destruir qualquer estrutura que possa ser criada.

Será criada uma _shared_key_, a ser usada no âmbito da cifra. 

In [4]:
def generateShared(prv_key_, peer_key):
    
    # Gerar uma chave partilha para cifra
    cipher_key = prv_key.exchange(peer_key)
    
    derived_key = HKDF(
        algorithm=hashes.SHA256(),
        length=32,
        salt=None,
        info=b'handshake data',
    ).derive(cipher_key)
    
    return cipher_key

## Assinar mensagem

A autenticação dos agentes é realizada com 
__“Ed448 Signing&Verification”__; sendo este um 
algoritmo de assinaturas com recurso ao __EdDSA__ (_Edwards-curve Digital Signature Algorithm_).

Para tal é necessário um novo par de chaves (privada/pública), do tipo _Ed448_, para cada um dos agentes.

Para recorrer aos meios apresentados em:
- https://cryptography.io/en/latest/hazmat/primitives/asymmetric/ed448/

As chaves públicas devem ser partilhadas por ambos, de 
modo a validar a __assinatura__ da mensagem que receberam.
Sendo a assinatura realizada através da chave privada do agente,
a enviar a mensagem.

Criou-se um método particular para gerar as chaves de autenticação dos agentes, para facilitar e tornar o código mais _readable_.

In [6]:
def generateSignKeys():
    
    ## Chave privada para assinar
    private_key = Ed448PrivateKey.generate()
    ## Chave pública para autenticar
    public_key = private_key.public_key()
    
    return private_key, public_key
    
def signMsg(prv_key, msg):
    
    signature = private_key.sign(msg)
    
    return signature

## Cifrar mensagem

In [ ]:
def encrypt:
    

## Decifrar mensagem

In [ ]:
def decrypt:
    

## Funções auxiliares para o canal de comunicação

In [10]:
def init_comm_emitter():
    ## Gerar chaves (cifrar e autenticar)
    prv_cipher_key, pub_cipher_key = generateKeys()
    prv_sign_key, pub_sign_key = generateSignKeys()
    
    ## Dicionário com a chavess públicas (serializadas)
    msg = {'cipher_key': pub_cipher_key.public_bytes(encoding=serialization.Encoding.PEM,
                                       format=serialization.PublicFormat.SubjectPublicKeyInfo
                                    ), 
           'sign_key': pub_sign_key.public_bytes(encoding=serialization.Encoding.PEM,
                                       format=serialization.PublicFormat.SubjectPublicKeyInfo
                                    )
            }
    
    return prv_cipher_key, pub_cipher_key, prv_mac_key, pub_mac_key, msg

def init_comm_receiver():
    ## Gerar chaves (cifrar e autenticar)
    prv_cipher_key, pub_cipher_key = generateKeys()
    
    ## Dicionário com a chavess públicas (serializadas)
    msg = {'cipher_key': pub_cipher_key.public_bytes(encoding=serialization.Encoding.PEM,
                                       format=serialization.PublicFormat.SubjectPublicKeyInfo
                                    )
          } 
    
    return prv_cipher_key, pub_cipher_key, msg

async def send(queue, msg):
    
    await asyncio.sleep(random.random())
        
    # put the item in the queue
    await queue.put(msg)
    
    await asyncio.sleep(random.random())
    
async def receive(queue):
    item = await queue.get()

    await asyncio.sleep(random.random())
    aux = loads(item)

    return aux

## Emitter

In [ ]:
## Emitter Code
async def emitter(plaintext, queue):
    
    ## Gerar as chaves (privada e publica) & partilhar com participante
    prv_cipher_key, pub_cipher_key, prv_sign_key, pub_sign_key, msg = init_comm_emitter()
    
    ## Enviar a chaves públicas para o peer
    await send(queue, dumps(msg))
    print("[E] SENDING PUBLIC KEYS")
    
    ## Receber as chaves públicas do peer
    msg = await receive(queue)
    print("[E] RECEIVED PEER PUBLIC KEYS")
    
    pub_peer_cipher = msg['cipher_key']
    print("[E] Receiver pub_key_cipher: " +str(msg['cipher_key']))
    
    ## Criar as chaves partilhadas (cifrar/autenticar)
    cipher_shared = generateShared(prv_cipher_key, pub_peer_cipher)
    
    ## Cifrar a mensagem
    pkg = encrypt(bytes(plaintext, 'utf-8'), cipher_shared, mac_shared)
    print("[E] MESSAGE ENCRYPTED")
    
    ## Assinar e enviar a mensagem
    pkg_b = dumps(pkg)
    sig = signMsg(prv_cipher_key, pkg_b)
    
    # Enviar
    msg_final = {'sig': sig, 'msg': dumps(pkg)}
    
    print("[E] SENDING MESSAGE")
    await send(queue, dumps(msg_final))
    
    print("[E] END")

## Receiver

In [11]:
## Receiver Code
async def receiver(queue):
    
    ## Gerar as chaves (privada e publica) & partilhar com participante
    prv_cipher_key, pub_cipher_key, msg = init_comm_receiver()
    
    ## Receber as chaves publicas do peer
    pub_keys = await receive(queue)
    
    pub_peer_cipher = pub_keys['cipher_key']
    pub_peer_sign = pub_keys['sign_key']
    
    print("[R] Emitter pub_key_cipher: " +str(pub_peer_cipher))
    print("[R] Emitter pub_key_sign: " +str(pub_peer_sign))
    
    ## Gerar shared keys
    cipher_shared = generateShared(prv_cipher_key, pub_peer_cipher)

    ## Enviar as chaves públicas ao peer
    await send(queue, dumps(msg))
    print("[R] AWAIT CIPHER")
    ciphertext = await receive(queue)
    print("[R] CIPHER RECEIVED")
    
    ## Receber a mensagem (Assinatura)
    peer_cipher_key = serialization.load_pem_public_key(pub_peer_cipher)
    
    ## Validar a correçaõ da assinatura
    peer_cipher_key.verify(ciphertext['sig'], ciphertext['msg'], ec.ECDSA(hashes.SHA3_256()))
       
    msg_dict = loads(ciphertext['msg'])
                      
    ## Decifrar essa mensagem       
    plain_text = decrypt(msg_dict['cipher'],msg_dict['HMAC'], cipher_shared, mac_shared, msg_dict['nounce'], msg_dict['ad'])
    
    ## Apresentar no terminal
    print("[R] Plaintext: " + plain_text.decode('utf-8'))
       

In [ ]:
def test(msg):
    loop = asyncio.get_event_loop()
    queue = asyncio.Queue(10)
    asyncio.ensure_future(emitter(msg, queue), loop=loop)
    loop.run_until_complete(receiver(queue))
    # emitter(msg)

test("HELLO WORLD!")